In [35]:
import pandas as pd 
import numpy as np
from scipy.stats import pearsonr
import networkx as nx
import matplotlib.pyplot as plt

Data

https://www.synapse.org/#!Synapse:syn2812961 

https://www.kaggle.com/murats/gene-expression-cancer-rnaseq?fbclid=IwAR3Mj1xpvD_sBBwn6ivsPf04pvWIm7n3QCKwSdb_bmxhFL2iEXvCGIParIs

In [4]:
LGG = pd.read_csv("unc.edu_LGG_IlluminaHiSeq_RNASeqV2.geneExp.tsv", sep='\t')
LUCS = pd.read_csv("unc.edu_LUSC_IlluminaHiSeq_RNASeqV2.geneExp.tsv", sep='\t')
PRAD = pd.read_csv("unc.edu_PRAD_IlluminaHiSeq_RNASeqV2.geneExp.tsv", sep = '\t')

Rows: Genes \
Columns: Patients

In [5]:
LGG.head(5)

,gene_id,TCGA-CS-4938-01B-11R-1896-07,TCGA-CS-4941-01A-01R-1470-07,TCGA-CS-4942-01A-01R-1470-07,TCGA-CS-4943-01A-01R-1470-07,TCGA-CS-4944-01A-01R-1470-07,TCGA-CS-5390-01A-02R-1470-07,TCGA-CS-5393-01A-01R-1470-07,TCGA-CS-5394-01A-01R-1470-07,TCGA-CS-5395-01A-01R-1470-07,...,TCGA-VW-A8FI-01A-11R-A36H-07,TCGA-W9-A837-01A-11R-A36H-07,TCGA-WH-A86K-01A-11R-A36H-07,TCGA-WY-A858-01A-11R-A36H-07,TCGA-WY-A859-01A-12R-A36H-07,TCGA-WY-A85A-01A-21R-A36H-07,TCGA-WY-A85B-01A-11R-A36H-07,TCGA-WY-A85C-01A-11R-A36H-07,TCGA-WY-A85D-01A-11R-A36H-07,TCGA-WY-A85E-01A-11R-A36H-07
0,?|100130426,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,?|100133144,8.7141,36.4493,11.8131,8.6116,0.000,5.3382,3.7845,8.3071,2.7732,...,19.1299,12.5770,10.9796,7.8225,3.3371,5.4526,8.2616,11.5121,5.3895,9.9433
2,?|100134869,22.7523,21.1767,11.0242,5.0835,30.261,27.8886,8.7230,15.4466,12.7776,...,21.9905,27.0125,14.2608,16.6975,10.2310,16.1890,19.7074,18.7291,8.9103,7.4480
3,?|10357,268.5760,156.6870,185.1380,269.8360,216.341,159.7590,198.1880,208.5410,255.2880,...,127.7470,106.8290,205.5170,184.4080,125.5470,124.9190,133.7390,120.9240,107.2540,143.0060
4,?|10431,845.8150,390.2690,621.4530,835.7290,812.508,576.9050,551.9470,607.8960,386.1780,...,564.3620,603.3720,570.9130,760.7160,875.1410,658.2210,662.6510,524.7240,705.1280,748.5820


In [6]:
LGG.shape

(20531, 535)

How to build a coexpression matrix?

https://link.springer.com/protocol/10.1007%2F978-1-4939-7747-5_21

In [31]:
# Pearson Correlation
def calculate_pearson(data, length):
    '''
    Input: Data sequence
    Output: Person matrix
    '''
    gene_ids = data['gene_id'][:length]
    n = len(gene_ids)
    pmatrix = np.empty((n, n))
    
    for i in range(n):
        for j in range(n):
            gene1 = list(data.iloc[i, :].values)[1:][:length]
            gene2 = list(data.iloc[j, :].values)[1:][:length]
            # note: if either one of gene is all zeros, r and p are nan
            r, p= pearsonr(gene1, gene2)
            pmatrix[i][j] = r
    return pmatrix

In [32]:
LGG_matrix = calculate_pearson(LGG, 100)

In [10]:
LGG_matrix.shape

(100, 100)

In [13]:
LGG_matrix

array([[ 1.00000000e+00, -1.98505370e-02,  2.46778066e-04, ...,
         5.00769019e-03, -5.16375120e-02,  7.29318583e-02],
       [-1.98505370e-02,  1.00000000e+00,  8.11636889e-01, ...,
         2.34167135e-02,  1.57924512e-01,  1.31143682e-02],
       [ 2.46778066e-04,  8.11636889e-01,  1.00000000e+00, ...,
        -4.27921068e-02, -1.78496181e-01, -9.99596869e-02],
       ...,
       [ 5.00769019e-03,  2.34167135e-02, -4.27921068e-02, ...,
         1.00000000e+00,  2.98234046e-01,  1.41670443e-01],
       [-5.16375120e-02,  1.57924512e-01, -1.78496181e-01, ...,
         2.98234046e-01,  1.00000000e+00,  9.99563196e-02],
       [ 7.29318583e-02,  1.31143682e-02, -9.99596869e-02, ...,
         1.41670443e-01,  9.99563196e-02,  1.00000000e+00]])

In [9]:
def write_to_file(filename, data):
    outfile = open(filename, 'wb')
    pickle.dump(data, outfile)
    outfile.close()


def load_pkl(filename):
    file = open(filename, 'rb')
    object_file = pickle.load(file)
    file.close()
    return object_file

In [33]:
# check for symmetric
def is_symmetric(matrix):
    n = len(matrix)
    for i in range(n):
        for j in range(n):
            if matrix[i][j] != matrix[j][i]:
                # can be nan
                if not np.isnan(matrix[i][j]) or not np.isnan(matrix[i][j]):
                    return False
    return True

In [34]:
is_symmetric(LGG_matrix)

True

In [79]:
def build_graph(data, adj_matrix, actual_name):
    # undirected, weighted graph
    G = nx.Graph()
    for i in range(len(adj_matrix)):
        if actual_name:
            G.add_node(data["gene_id"].iloc[i])
        else:
            G.add_node(i)
        for j in range(i+1, len(adj_matrix)):
            if not np.isnan(adj_matrix[i][j]): 
                if actual_name:
                    G.add_edge(data["gene_id"].iloc[i], data["gene_id"].iloc[j], weight=adj_matrix[i][j])
                else:
                    G.add_edge(i, j, weigt=adj_matrix[i][j])
    return G
                                        

In [82]:
G = build_graph(LGG, LGG_matrix, False)

In [102]:
# test to check graph is correct
print ("Number of nodes: ", nx.number_of_nodes(G) == len(LGG_matrix))
print ("Number of selfloop: ", nx.number_of_selfloops(G))
print (G.get_edge_data(0,1), LGG_matrix[0][1])
print (G.get_edge_data(2, 4), LGG_matrix[2][4])

Number of nodes:  True
Number of selfloop:  0
{'weigt': -0.019850536952359717} -0.019850536952359717
{'weigt': 0.21036652906343162} 0.21036652906343162


In [104]:
print ("nodes number: {}, edges number: {}".format(nx.number_of_nodes(G), nx.number_of_edges(G)))
print ("Number of connected components: ", nx.number_connected_components(G))

nodes number: 100, edges number: 4278
Number of connected components:  8


In [111]:
def analyze_connected_component(graph):
    largest_connected_component = max(nx.connected_component_subgraphs(graph), key=len)
    num_nodes = len(largest_connected_component.nodes)
    degree = largest_connected_component.degree() 
    return largest_connected_component, num_nodes, degree

In [112]:
largest_cc, num_nodes, degree = analyze_connected_component(G)

In [116]:
print ("largest cc num of nodes: ", num_nodes)

largest cc num of nodes:  93
